In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('../data/preprocessed_train.csv', encoding='utf-8')
df.head()

,title,author,text,label,removed_punc,tokens,filtered_tokens,clean_tokens,lemma_words,clean_text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide We Didn’t Even See Comey’s Lett...,"['house', 'dem', 'aide', 'we', 'didn’t', 'even...","['house', 'aide', 'didn’t', 'even', 'comey’s',...","['house', 'aide', 'didn’t', 'even', 'comey’s',...","['house', 'aide', 'didn’t', 'even', 'comey’s',...",house aide didn’t even comey’s letter jason ch...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,Ever get the feeling your life circles the rou...,"['ever', 'get', 'the', 'feeling', 'your', 'lif...","['ever', 'feeling', 'your', 'life', 'circles',...","['ever', 'feeling', 'life', 'circles', 'rounda...","['ever', 'feeling', 'life', 'circle', 'roundab...",ever feeling life circle roundabout rather hea...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired October 29 2...,"['why', 'the', 'truth', 'might', 'get', 'you',...","['truth', 'might', 'fired', 'october', '2016',...","['truth', 'might', 'fired', 'october', '2016',...","['truth', 'might', 'fired', 'october', '2016',...",truth might fired october 2016 tension intelli...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Videos 15 Civilians Killed In Single US Airstr...,"['videos', '15', 'civilians', 'killed', 'in', ...","['videos', 'civilians', 'killed', 'single', 'a...","['videos', 'civilians', 'killed', 'single', 'a...","['video', 'civilian', 'killed', 'single', 'air...",video civilian killed single airstrike identif...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Print \nAn Iranian woman has been sentenced to...,"['print', 'an', 'iranian', 'woman', 'has', 'be...","['print', 'iranian', 'woman', 'been', 'sentenc...","['print', 'iranian', 'woman', 'sentenced', 'ye...","['print', 'iranian', 'woman', 'sentenced', 'ye...",print iranian woman sentenced year prison iran...


In [14]:
data = df.loc[df['label'] == 1, 'clean_text']
data

0        house aide didn’t even comey’s letter jason ch...
2        truth might fired october 2016 tension intelli...
3        video civilian killed single airstrike identif...
4        print iranian woman sentenced year prison iran...
10       mystery surrounding third reich nazi germany s...
                               ...                        
18229    google pinterest digg linkedin reddit stumbleu...
18232    lawyer kept hillary campaign chief jail hillar...
18234    share although vandal thought “cool” destroy d...
18239    nato russia hold parallel exercise balkan 1102...
18240    david swanson author activist journalist radio...
Name: clean_text, Length: 7880, dtype: object

# Generator with LSTM

In [15]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Here we should be reading from the FAKE NEWS
texts = data.tolist()
texts = [text for text in texts if not (isinstance(text, float) and np.isnan(text))]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
total_words = len(tokenizer.word_index) + 1  # Vocabulary size

input_sequences = []
for line in texts:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X = input_sequences[:, :-1]
y = input_sequences[:, -1]

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length - 1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train your model using X and y
# You might want to one-hot encode y for categorical crossentropy

# Generate text using the trained model
seed_text = "Your starting text here"
next_words = 100

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word = tokenizer.index_word[np.argmax(predicted)]
    seed_text += " " + predicted_word

print(seed_text)

# Generator with Transformers

In [8]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, AdamW

# Here we should be reading from the FAKE NEWS
text_data = [
    "Fake news sentence 1",
    "Fake news sentence 2",
    "Fake news sentence 3",
    # Add more fake news sentences as needed
]

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenized_data = tokenizer(text_data, truncation=True, max_length=512, return_tensors='pt')

# Model Training
config = GPT2Config.from_pretrained('gpt2')
model = GPT2LMHeadModel(config)
train_dataset = torch.utils.data.TensorDataset(tokenized_data['input_ids'], tokenized_data['attention_mask'])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
optimizer = AdamW(model.parameters(), lr=1e-5)

num_epochs = 5
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids, attention_mask = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Generate text using the trained model
seed_text = "give a title for a news article"
input_ids = tokenizer.encode(seed_text, return_tensors='pt')
output = model.generate(input_ids, max_length=200, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Fake News: ", generated_text)


d:\programas\programas\github repos\nlp\nlp\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Generated Fake News:  Your starting text here sentence sentence sentence sentence sentence sentence sentence sentence sentence sentence 3 sentence sentence 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 1 1 1 1 1 sentence sentence sentence 3 3 3 3 3 3 3 3 3 3 3 3 3 3 sentence 1 1 1 1 1 1 1 sentence sentence sentence sentence sentence 1 1 1 sentence sentence 1 1 1 1 1 1 1 1 1 1 1 sentence sentence sentence sentence sentence sentence sentence sentence sentence sentence 3 sentence sentence sentence sentence sentence sentence 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 sentence sentence sentence 3 3 3 3 3 3 sentence sentence sentence 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 sentence sentence sentence sentence sentence sentence sentence sentence sentence sentence sentence 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
